In [22]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [23]:
df=pd.read_csv("diabetes.csv")
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [24]:
Col_name=["Pregnancies","Glucose","BloodPressure","SkinThickness",
         "Insulin","BMI","DiabetesPedigreeFunction","Age","Class"]

In [25]:
df=pd.read_csv("diabetes.csv", header=0, names=Col_name)
df.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


In [26]:
df.head(0)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class


In [27]:
#normalise using custom lambda function
col_norm=["Pregnancies","Glucose","BloodPressure","SkinThickness",
         "Insulin","BMI","DiabetesPedigreeFunction"]
df_norm=df[col_norm]

In [28]:
df_norm.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction
0,6,148,72,35,0,33.6,0.627
1,1,85,66,29,0,26.6,0.351


In [29]:
dfl_norm=df[col_norm].apply(lambda x:((x-x.min())/(x.max()-x.min())))

In [30]:
dfl_norm.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction
0,0.352941,0.743719,0.590164,0.353535,0.0,0.500745,0.234415
1,0.058824,0.427136,0.540984,0.292929,0.0,0.396423,0.116567


In [31]:
feat_Pregnancies=tf.feature_column.numeric_column("Pregnancies")
feat_Glucose=tf.feature_column.numeric_column("Glucose")
feat_BloodPressure=tf.feature_column.numeric_column("BloodPressure")
feat_SkinThickness_tricep=tf.feature_column.numeric_column("SkinThickness")
feat_Insulin=tf.feature_column.numeric_column("Insulin")
feat_BMI=tf.feature_column.numeric_column("BMI")
feat_DiabetesPedigreeFunction=tf.feature_column.numeric_column("DiabetesPedigreeFunction")

In [32]:
feat_cols=[feat_Pregnancies,feat_Glucose,feat_BloodPressure,feat_SkinThickness_tricep,feat_Insulin,feat_BMI,feat_DiabetesPedigreeFunction]

In [33]:
X_Data=dfl_norm
X_Data.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction
0,0.352941,0.743719,0.590164,0.353535,0.0,0.500745,0.234415
1,0.058824,0.427136,0.540984,0.292929,0.0,0.396423,0.116567


In [34]:
Y_Data=df["Class"]
Y_Data.head(3)

0    1
1    0
2    1
Name: Class, dtype: int64

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X_Data, Y_Data, test_size=0.3, random_state=101)

In [36]:
input_func=tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train, y=Y_train, batch_size=10,num_epochs=100,shuffle=True)

In [37]:
test_func=tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test, y=Y_test, batch_size=10,num_epochs=1,shuffle=False)

In [38]:
model=tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\hafiz\\AppData\\Local\\Temp\\tmpa_6d7uy9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [39]:
model.train(input_fn=input_func, steps=5000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\hafiz\AppData\Local\Temp\tmpa_6d7uy9\model.ckpt.
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 144.081
INFO:tensorflow:loss = 0.6028807, step = 100 (0.696 sec)
INFO:tensorflow:global_ste

In [40]:
eval_input_func=tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test, y=Y_test, batch_size=10,num_epochs=1,shuffle=False)

In [41]:
results=model.evaluate(eval_input_func)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-27T15:33:16Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\hafiz\AppData\Local\Temp\tmpa_6d7uy9\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 1.90566s
INFO:tensorflow:Finished evaluation at 2020-03-27-15:33:18
INFO:tensorflow:Saving dict for global step 5000: 

In [42]:
results["accuracy"]

0.7835498